In [37]:
import pandas as pd
import itertools
df = pd.read_csv('D:/Study-3rd/DSP/OP-heist-project/data/combined/trade_by_region_industry_direction_breakdown(employment).csv',encoding='iso-8859-1')

In [38]:
mapping = {
    '10-12 Food industry etc.': 'C Manufacturing (10-33)',
    '13-15 Textile, clothing and leather industry': 'C Manufacturing (10-33)',
    '16 Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials': 'C Manufacturing (10-33)',
    '17, 18 Paper industry; Printing': 'C Manufacturing (10-33)',
    '19-22 Chemical industry': 'C Manufacturing (10-33)',
    '23 Manufacture of other non-metallic mineral products': 'C Manufacturing (10-33)',
    '24-25 Manufacture of basic metals and fabricated metal products, except machinery and equipment': 'C Manufacturing (10-33)',
    '26, 27 Manufacture of electrical and electronic products': 'C Manufacturing (10-33)',
    '28 Manufacture of machinery and equipment n.e.c.': 'C Manufacturing (10-33)',
    '29, 30 Manufacture of transport equipment': 'C Manufacturing (10-33)',
    '31-33 Manufacture of furniture, other manufacturing; repair and installation of machinery and equipment': 'C Manufacturing (10-33)',
    '681, 68209, 683 Other real estate activities': 'L Real estate activities (68)',
    '68201, 68202 Letting and operation of dwellings': 'L Real estate activities (68)'
}
df['Industry'] = df['Industry'].map(lambda x: mapping.get(x, x))
df1 = df.groupby(['Time', 'Area', 'Industry', 'Direction'])['Trade value'].sum().reset_index()
df2 = df1.groupby(['Time','Area','Direction'])['Trade value'].sum().reset_index()

In [39]:
permutations = pd.merge(df1, df2, left_on=['Time', 'Area', 'Direction'],right_on=['Time', 'Area', 'Direction'], how='left')
permutations.rename(columns = {'Trade value_x':'Trade value (euro)','Trade value_y':'Region total trade value (euro)'},inplace = True)
permutations['Time'] = permutations['Time'].astype(str).str[0:4]
permutations['Area'] = permutations['Area'].astype(str).str[0:4]
permutations = permutations.pivot_table(index = ['Time','Area','Industry'],columns = 'Direction',values = ['Trade value (euro)'])
permutations
permutations.reset_index(inplace=True)
permutations.columns = ['Time', 'Area', 'Industry', 'Export', 'Import']

In [40]:
df_gdp = pd.read_csv('D:/Study-3rd/DSP/OP-heist-project/data/Regional_economic_and_environmental_data/gdp_region.csv',encoding = 'iso-8859-1')
df_gdp['Area'] = df_gdp['Area'].astype(str).str[0:4]
df_gdp['2015'] = df_gdp['2015 At current prices, euro'] * df_gdp['2015 Population']
df_gdp['2016'] = df_gdp['2016 At current prices, euro'] * df_gdp['2016 Population']
df_gdp['2017'] = df_gdp['2017 At current prices, euro'] * df_gdp['2017 Population']
df_gdp['2018'] = df_gdp['2018 At current prices, euro'] * df_gdp['2018 Population']
df_gdp['2019'] = df_gdp['2019 At current prices, euro'] * df_gdp['2019 Population']
df_gdp['2020'] = df_gdp['2020 At current prices, euro'] * df_gdp['2020 Population']
df_gdp = df_gdp.drop(columns=df_gdp.columns[1:13])
df_gdp = df_gdp.melt(id_vars = 'Area',var_name = 'Time', value_name = 'Total GDP')
permutations.reset_index(inplace=True)
permutations_new = pd.merge(permutations, df_gdp, on=['Area', 'Time'], how='left')
industries = permutations_new
regions = permutations_new

In [41]:
industries['Industry Trade Dependency'] = (industries['Export'] + industries['Import'])/industries['Total GDP'] * 100
industries['Import-Export Imbalance'] = (industries['Import'] - industries['Export'])/(industries['Import'] + industries['Export']) * 100
industries = industries.drop(columns = 'index')
industries['Time'] = industries['Time'].astype(int)
years = industries['Time'].unique()
industries_unique = industries['Industry'].unique()
areas = industries['Area'].unique()

for year, indust, area in itertools.product(list(years), list(industries_unique), list(areas)):
    if year == 2015:
        continue  # Skip the first year

    mask_current = (industries['Time'] == year) & (industries['Industry'] == indust) & (industries['Area'] == area)
    mask_prev = (industries['Time'] == year - 1) & (industries['Industry'] == indust) & (industries['Area'] == area)

    curr_exp = industries[mask_current]['Export'].values[0]
    curr_imp = industries[mask_current]['Import'].values[0]
    prev_exp = industries[mask_prev]['Export'].values[0]
    prev_imp = industries[mask_prev]['Import'].values[0]

    export_growth = (curr_exp - prev_exp) / prev_exp * 100
    import_growth = (curr_imp - prev_imp) / prev_imp * 100

    exp_growth_abs = curr_exp - prev_exp
    imp_growth_abs = curr_imp - prev_imp
    industries.loc[mask_current, 'Export Growth'] = export_growth
    industries.loc[mask_current, 'Import Growth'] = import_growth
    industries.loc[mask_current, 'Export Growth (Absolute)'] = exp_growth_abs
    industries.loc[mask_current, 'Import Growth (Absolute)'] = imp_growth_abs

industries = industries.fillna(0)

In [42]:
regions = regions.groupby(['Time','Area']).sum()
regions = regions.drop(columns = ['Industry','Industry Trade Dependency','Import-Export Imbalance','index'])
regions['Total GDP'] = regions['Total GDP']/18
regions['Regional Trade Dependency'] = (regions['Export'] + regions['Import'])/regions['Total GDP']*100
regions['Regional Import-Export Imbalance'] = (regions['Import'] - regions['Export'])/(regions['Import'] + regions['Export'])*100
regions.reset_index(inplace=True)
regions.columns = ['Time', 'Area', 'Regional export', 'Regional import', 'Total GDP', 'Regional Trade Dependency','Regional Import-Export Imbalance']
regions['Time'] = regions['Time'].astype(int)
years = regions['Time'].unique()
areas = regions['Area'].unique()
for year, area in itertools.product(list(years), list(areas)):
    if year == 2015:
        continue

    mask_current = (regions['Time'] == year) & (regions['Area'] == area)
    mask_prev = (regions['Time'] == year - 1) & (regions['Area'] == area)

    curr_exp = regions[mask_current]['Regional export'].values[0]
    curr_imp = regions[mask_current]['Regional import'].values[0]
    prev_exp = regions[mask_prev]['Regional export'].values[0]
    prev_imp = regions[mask_prev]['Regional import'].values[0]

    export_growth = (curr_exp - prev_exp) / prev_exp * 100
    import_growth = (curr_imp - prev_imp) / prev_imp * 100
    exp_growth_abs = curr_exp - prev_exp
    imp_growth_abs = curr_imp - prev_imp

    regions.loc[mask_current, 'Regional Export Growth'] = export_growth
    regions.loc[mask_current, 'Regional Import Growth'] = import_growth
    regions.loc[mask_current, 'Export Growth (Absolute)'] = exp_growth_abs
    regions.loc[mask_current, 'Import Growth (Absolute)'] = imp_growth_abs
    
regions = regions.fillna(0)

In [43]:
overall = pd.merge(industries,regions,left_on=['Time','Area'],right_on=['Time','Area'],how='left')
overall.rename(columns={'Total GDP_y':'Total GDP',
                        'Export':'Industry export','Import':'Industry import',
                        'Import-Export Imbalance':'Industry Import-Export Imbalance',
                        'Export Growth':'Industry Export Growth',
                        'Import Growth':'Industry Import Growth',
                        'Export Growth (Absolute)_y':'Regional Export Growth (Absolute)',
                        'Import Growth (Absolute)_y':'Regional Import Growth (Absolute)'
                        },inplace = True)
overall['Export concentration'] = overall['Industry export']/overall['Regional export']*100
grouped_data = overall.groupby(['Time', 'Area'])
result_df = pd.DataFrame()

for (time, area), group in grouped_data:
    max_export_concentration = group['Export concentration'].max()
    max_export_rows = group[group['Export concentration'] == max_export_concentration]
    result_df = pd.concat([result_df, max_export_rows])
# Some cleaning
result_df = result_df.drop(columns=result_df.columns[3:12])
result_df = result_df.drop(columns='Total GDP')
result_df['Industry'] = result_df['Industry'].astype(str).str[0]
result_df = result_df.rename(columns = {'Industry':'Dominant Industry'})
result_df.reset_index(inplace=True)
result_df = result_df.drop(columns='index')
result_df.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/Indices/regional_indexes.csv')
industries['Industry'] = industries['Industry'].astype(str).str[0]
industries = industries.drop(columns = industries.columns[5])
industries.to_csv('D:/Study-3rd/DSP/OP-heist-project/data/Indices/industries_indexes.csv')